In [0]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import output
from IPython import display
import random
import copy

# 1. Entendiendo el algoritmo
![Las partes del algoritmo](https://i.imgur.com/EwN2C8L.png)


*   __Jugador:__ que dado un estado del juego, toma la mejor opcion dependiendo de su estrategia.
*   __Juego:__ establece las reglas del juego, diciendole a los jugadores cuales son las acciones posibles, cuando termina el juego y si hay un ganador.
* __MonteCarlo:__ repite múltiplos juego para mejorar la estrategia de uno de los jugadores.



# 1. El Jugador

In [0]:
class jugador: 
  def __init__(self, valor_estado_accion,epsilon = 1):
    """
    Representa a un jugador que decide tomar una accion de acuerdo al estado 
    para maximizar su función de valor
    
    @param[in] valor_estado_accion: Arreglo de tamaño (3,3,3,3,3,3,3,3,3,9), 
               donde las primeras casillas especifican el estado y la ultima la 
               acción a tomar. Se asume que 1 corresponde a una ficha del otro
               jugador y 2 corresponde a una ficha de este jugador y 0 a una 
               ficha vacia
               
    @param[in] epsilon: es el componente aleatorio en la seleccion de la mejor
               accion. Debe ser positivo. Por defecto se asume totalmente
               aleatorio (epsilon = 1)
               
    @member    valor_estado_accion
    
    @member    epsilon
    """
    self.valor_estado_accion = copy.deepcopy(valor_estado_accion)
    self.epsilon = epsilon
  
  
  def accion_optima(self, estado, posibles_acciones):
    """
    Esta función recibe un estado y una lista de posibles acciones y retorna la 
    accion optima para el jugador, usando el valor_estado_accion y el azar.
    
    @param[in] estado: Lista de 9 numeros con valores en las casillas en [0,1,2]
               donde 1 corresponde a una ficha del otro
               jugador y 2 corresponde a una ficha de este jugador y 0 a una 
               ficha vacia
               
    @param[in] posibles_acciones: Lista de numeros del 0 al 8 de casillas que 
               están vacias. Se asume que no está vacia
                              
    @return    mejor_accion: Numero del 0 al 8 que identifica la accion optima 
               de este jugador, que pertenece al vecotr de posibles acciones
    """
     
    #1. Decidir si se toma la accion al azar o se optimiza
    # Para esto generamos un número aleatorio. Si menor a epsilon * aleatoriedad
    # la decision es al azar, si no encontramos la mejor en valor_acciones
      
    u = random.random()
    
    #2. Si es al azar,  
    if u < self.epsilon:
      #3.1 Tome una muestra de posibles_acciones
      mejor_accion = random.choice(posibles_acciones)
    
    #3. Si no, 
    else:
      #4.1 Extraer la fila de valor_estado_accion que corresponde a estado 
      # y que contiene solo las acciones posibles
      valor_acciones = self.valor_estado_accion[tuple(estado)] [posibles_acciones]

      #4.2 Encuentre el indice con mayor valor en la fila que extraimos en 1.
      mejor_valor = np.amax(valor_acciones)
      mejores_indices = np.where(valor_acciones == mejor_valor )[0]
      
      #4.3 Traducir este indice en el vector de acciones posibles 
      mejor_accion = posibles_acciones[random.choice(mejores_indices)] 
      
    #4. Retornar la mejor accion
    return mejor_accion


# 2. El Juego

In [0]:
class juego:
  def __init__(self, jugador1, jugador2, quien_inicia = 1):
    """
    Crea la infraestructura para manejar un juego donde optimizamos jugador2
    
    @param[in] jugador1: Objeto de la clase jugador con índice 1
    
    @param[in] jugador2: Objeto de la clase jugador con índice 2
                      
    @member    jugador1
    
    @member    jugador2  
    
    @member    lista_estados: Lista de estados para jugador2
    
    @member    lista_acciones: Lista de acciones para jugador2
    
    """
    
    self.jugador1 = jugador1
    self.jugador2 = jugador2
    self.quien_inicia = quien_inicia
    #1. Definir estado inicial, todas las casillas vacias
    self.estado = [0]*9
    #2. Definir un boolean para saber cuando termina el juego
    self.fin_juego = False
    #3. Definir una lista de estados vacia
    self.lista_estados =[]
    #4. Definir una lista de acciones vacia
    self.lista_acciones =[]
    self.ganador = 0
  
  def estado_conjugado(self):
    """
    Intercambia 1 y 2 para que el jugador1 pueda ver el estado como necesita
    
    @return nuevo_estado: Estado visto desde los ojos del jugador1 donde 2 es
            para el y 1 es para el otro jugador
    """
    nuevo_estado = []
    for casilla in self.estado:
      
      if casilla == 1:
        nuevo_estado.append(2)
      elif casilla == 2:
        nuevo_estado.append(1)
      else:
        nuevo_estado.append(0)
        
    return nuevo_estado
  
  def actualizar_ganador(self):
    """
    Esta función decide si en una configuracion hay un ganador
    @param[in] estado   Vector con entradas de 0, 1 o 2 que define la 
                        configuración actual
    @return    ganador  Regresa 0 si no hay ganador, 1 si X es ganador y 2 si O 
                        es ganador 
    """
    #Reorganizar la configuracion como el tablero de triqui
    matriz_estado = np.reshape(np.array(self.estado),(3,3))

    # 1. Revisar filas
    for i in xrange(3):
      #Si la fila no esta vacia
      if (matriz_estado[i,0] > 0):
      
        #Verificar que las 3 entradas de la fila son iguales
        if ((matriz_estado[i,0] == matriz_estado[i,1]) and 
            (matriz_estado[i,0] == matriz_estado[i,2])):
          self.ganador = matriz_estado[i,0]
          self.fin_juego = True
          return 
  
    # 2. Revisar Columnas
    for i in xrange(3):
      #Si la columna no esta vacia
      if (matriz_estado[0,i] > 0):
      
        #Verificar que las 3 entradas de la fila son iguales
        if ((matriz_estado[0,i] == matriz_estado[1,i]) and 
            (matriz_estado[0,i] == matriz_estado[2,i])):
          self.ganador = matriz_estado[0,i]
          self.fin_juego = True
          return  
      
    # 3. Revisar diagonal principal
    #Si la diagonal no esta vacia
    if (matriz_estado[0,0] > 0):
      #Verificar que las 3 entradas de la fila son iguales
      if ((matriz_estado[0,0] == matriz_estado[1,1]) and 
          (matriz_estado[0,0] == matriz_estado[2,2])):
        self.ganador = matriz_estado[0,0]
        self.fin_juego = True
        return  
    
    # 4. Revisar diagonal secundaria
    #Si la diagonal no esta vacia
    if (matriz_estado[0,2] > 0):
      #Verificar que las 3 entradas de la fila son iguales
      if ((matriz_estado[0,2] == matriz_estado[1,1]) and 
          (matriz_estado[0,2] == matriz_estado[2,0])):
        self.ganador = matriz_estado[0,2]
        return 
  
    #Si llegamos a este punto es que no hay ganador
    return None
  
  def turno_jugador1(self):
    """
    Aplica accion del jugador 1 y modifica el estado incluyendo esta acción
    @pre Se asume que hay posibles acciones, i.e. len(posibles_acciones)>0
    @post self.estado cuenta con la ultima accion de jugador 1
    """
    #1. Encontrar que casillas estan vacias
    posibles_acciones = np.where( np.array(self.estado) == 0 )[0]
    
    #2. Verificar que es posible moverse
    if len(posibles_acciones)<1:
      #2.1 Si no se puede mover, es el fin del juego
      self.fin_juego = True
      return None
    
    #2. Cambiar Estado para jugador 1
    estado1 = self.estado_conjugado()
    
    #4. Obtener optima accion del jugador 1 
    accion = self.jugador1.accion_optima(estado1, posibles_acciones)
    #5. Modificar estado del juego
    self.estado[accion] = 1
    
  def turno_jugador2(self):
    """
    Aplica accion del jugador 2 y modifica el estado incluyendo esta acción
    @pre Se asume que hay posibles acciones, i.e. len(posibles_acciones)>0
    @post self.estado cuenta con la ultima accion de jugador 2
    """
    #1. Encontrar que casillas estan vacias
    posibles_acciones = np.where( np.array(self.estado) == 0 )[0]
        
    #2. Verificar que es posible moverse
    if len(posibles_acciones)<1:
      #2.1 Si no se puede mover, es el fin del juego
      self.fin_juego = True
      return  
    
    #4. Obtener optima accion del jugador 1 
    accion = self.jugador2.accion_optima(self.estado, posibles_acciones)
    
    #5. Adicionar estado actual a la lista de estados
    self.lista_estados.append(copy.deepcopy(self.estado))
    
    #6. Adicionar accion a la lista de acciones
    self.lista_acciones.append(accion)
    
    #5. Modificar estado del juego
    self.estado[accion] = 2
    
    
  def jugar(self):
    """
    Ejecuta una partida de triqui
    
    @param[in] quien_inicia: Numero 1 o 2 para decidir que jugador empieza
    
    @return ganador: 0 si es empate, 1 si es jugador1, 2 si es jugador 2
    """
    
    
    #1. Si jugador 1 no comienza, entonces jugador 2 comienza
    if not (self.quien_inicia == 1):
      self.turno_jugador2()
      self.actualizar_ganador()
    
    #2. Iterar hasta que sea el fin del juego 
    while not self.fin_juego:
      #2.1 Turno jugador1 y actualizar ganado
      self.turno_jugador1()
      self.actualizar_ganador()
      
      if self.fin_juego:
        break;
      
      self.turno_jugador2()
      self.actualizar_ganador()
       
    return [self.ganador,self.lista_estados,self.lista_acciones]
      
      

# 3. La optimizacion del juego 

In [0]:
class MonteCarlo:
  def __init__(self, jugador_estatico, jugador_dinamico, gamma = 0.8):
    self.jugador_estatico = jugador_estatico
    self.jugador_dinamico = jugador_dinamico
    self. gamma = gamma
    
  def optimizar_jugador_dinamico(self,iteraciones = 1000, epsilon = 100, alpha = 100):
    #Iterar sobre el total de iteraciones
    for i in xrange(iteraciones):
      
      #1. Decidir quien inicia
      quien_inicia = random.choice([1,2])
      
      #2.Modificar azar jugador dinamico
      self.jugador_dinamico.epsilon = epsilon / (1. + i)
      
      #3. Crear un juego nuevo
      nuevo_juego = juego(self.jugador_estatico, self.jugador_dinamico, quien_inicia)
    
      #4. Ejercer un juego
      [ganador,estados,acciones] = nuevo_juego.jugar()
      
      precio = 0
    
      if ganador == 2:
        precio = 1
      elif ganador == 1:
        precio = -1
             
      for paso,(estado,accion) in enumerate(zip(estados,acciones)):
        indice = tuple(estado + [accion,])
        exponente = len(acciones) - paso
        error = precio * (self.gamma ** exponente) - self.jugador_dinamico.valor_estado_accion[indice] 
        self.jugador_dinamico.valor_estado_accion[indice] += error/alpha

    
    

# 4. Primera Optimizacion

In [0]:
#Crear jugador estatico como aleatorio
jugador_estatico = jugador([],epsilon = 1)

#Crear un valor estado_accion vacio
valor_estado_accion = np.zeros((3,)*9+(9,))

#Crear jugador dinamico
jugador_dinamico = jugador(valor_estado_accion,epsilon = 1)

#Definir factor descuento
gamma = 0.8

#Crear Monte Carlo
MC = MonteCarlo(jugador_estatico, jugador_dinamico, gamma)

#Definir Iteraciones
iteraciones = 1000

#Aplicar Monte Carlo
MC.optimizar_jugador_dinamico(iteraciones=100000, epsilon = 100, alpha = 100)

#5. Jugador Interactivo

In [0]:
class juego_interactivo:
  def __init__(self, jugador2):
    """
    Crea la infraestructura para manejar un juego donde optimizamos jugador2
        
    @param[in] jugador2: Objeto de la clase jugador con índice 2
                         
    @member    jugador2  
       
    """
    
    self.jugador2 = jugador2
    #1. Definir estado inicial, todas las casillas vacias
    self.estado = [0]*9
    self.inicio_juego = False
    #2. Definir un boolean para saber cuando termina el juego
    self.fin_juego = False
    self.ganador = 0
  
  def comenzar_juego(self):
    if not self.inicio_juego:
      print("Yo comienzo :)")
      self.turno_jugador2()

    
  def actualizar_ganador(self):
    """
    Esta función decide si en una configuracion hay un ganador
    @param[in] estado   Vector con entradas de 0, 1 o 2 que define la 
                        configuración actual
    @return    ganador  Regresa 0 si no hay ganador, 1 si X es ganador y 2 si O 
                        es ganador 
    """
    #Reorganizar la configuracion como el tablero de triqui
    matriz_estado = np.reshape(np.array(self.estado),(3,3))

    # 1. Revisar filas
    for i in xrange(3):
      #Si la fila no esta vacia
      if (matriz_estado[i,0] > 0):
      
        #Verificar que las 3 entradas de la fila son iguales
        if ((matriz_estado[i,0] == matriz_estado[i,1]) and 
            (matriz_estado[i,0] == matriz_estado[i,2])):
          self.ganador = matriz_estado[i,0]
          self.fin_juego = True
          self.print_ganador()
          return 
  
    # 2. Revisar Columnas
    for i in xrange(3):
      #Si la columna no esta vacia
      if (matriz_estado[0,i] > 0):
      
        #Verificar que las 3 entradas de la fila son iguales
        if ((matriz_estado[0,i] == matriz_estado[1,i]) and 
            (matriz_estado[0,i] == matriz_estado[2,i])):
          self.ganador = matriz_estado[0,i]
          self.fin_juego = True
          self.print_ganador()
          return  
      
    # 3. Revisar diagonal principal
    #Si la diagonal no esta vacia
    if (matriz_estado[0,0] > 0):
      #Verificar que las 3 entradas de la fila son iguales
      if ((matriz_estado[0,0] == matriz_estado[1,1]) and 
          (matriz_estado[0,0] == matriz_estado[2,2])):
        self.ganador = matriz_estado[0,0]
        self.fin_juego = True
        self.print_ganador()
        return  
    
    # 4. Revisar diagonal secundaria
    #Si la diagonal no esta vacia
    if (matriz_estado[0,2] > 0):
      #Verificar que las 3 entradas de la fila son iguales
      if ((matriz_estado[0,2] == matriz_estado[1,1]) and 
          (matriz_estado[0,2] == matriz_estado[2,0])):
        self.ganador = matriz_estado[0,2]
        self.fin_juego = True
        self.print_ganador()
        return 
  
    #Si llegamos a este punto es que no hay ganador
    return 
  
  def turno_jugador1(self,accion):
    """
    Aplica accion del jugador 1 y modifica el estado incluyendo esta acción
    @pre Se asume que hay posibles acciones, i.e. len(posibles_acciones)>0
    @post self.estado cuenta con la ultima accion de jugador 1
    """
    self.inicio_juego = True
    if self.fin_juego:
        return
    
    posibles_acciones = np.where( np.array(self.estado) == 0 )[0]
    
    #2. Verificar que es posible moverse
    if len(posibles_acciones)<1:
      #2.1 Si no se puede mover, es el fin del juego
      self.fin_juego = True
      self.print_ganador()
      return
    
    
    #1. Verificar accion posible
    if self.estado[accion]>0:
      return
    
    
    #5. Modificar estado del juego
    self.estado[accion] = 1
    self.imprimir()
    self.actualizar_ganador()
    self.turno_jugador2()
    
  def turno_jugador2(self):
    """
    Aplica accion del jugador 2 y modifica el estado incluyendo esta acción
    @pre Se asume que hay posibles acciones, i.e. len(posibles_acciones)>0
    @post self.estado cuenta con la ultima accion de jugador 2
    """
    if self.fin_juego:
        return
    
    #1. Encontrar que casillas estan vacias
    posibles_acciones = np.where( np.array(self.estado) == 0 )[0]
        
    #2. Verificar que es posible moverse
    if len(posibles_acciones)<1:
      #2.1 Si no se puede mover, es el fin del juego
      self.fin_juego = True
      self.print_ganador()
      return  
    
    #4. Obtener optima accion del jugador 1 
    accion = self.jugador2.accion_optima(self.estado, posibles_acciones)

    #5. Modificar estado del juego
    self.estado[accion] = 2
    self.imprimir()
    self.actualizar_ganador()
    
  def imprimir(self):
    """
    Esta funcion imprime el tablero en forma de matriz
    """
    for i in xrange(3):
      linea = " ";
      for j in xrange(3):
        if self.estado[i*3+j] == 0:
          linea += " "
        if self.estado[i*3+j] == 1:
          linea += "X"
        if self.estado[i*3+j] == 2:
          linea += "O"
        if j < 2:
          linea += "|"
      print(linea)
    print("-+-+-+-+-+")
    
  def print_ganador(self):
    if self.ganador == 1:
      print("Tu ganaste")
    elif self.ganador == 2:
      print("Yo gane")
    else:
      print("Empate: No hay ganador")

In [24]:
jugador_dinamico.valor_estado_accion[tuple([0]*9)]

array([0.06749877, 0.03630051, 0.07235785, 0.03683401, 0.08425518,
       0.35710625, 0.06808279, 0.08804519, 0.03036597])

In [36]:
display.display(display.HTML('''
    Posiciones del juego, tu eres X, yo soy O: <br>
    <button id='button0'>0</button>
    <script>
      document.querySelector('#button0').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [0], {});
      };
    </script>
    <button id='button1'>1</button>
    <script>
      document.querySelector('#button1').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [1], {});
      };
    </script>
    <button id='button2'>2</button>
    <script>
      document.querySelector('#button2').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [2], {});
      };
    </script><br>
    <button id='button3'>3</button>
    <script>
      document.querySelector('#button3').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [3], {});
      };
    </script>
    <button id='button4'>4</button>
    <script>
      document.querySelector('#button4').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [4], {});
      };
    </script>
    <button id='button5'>5</button>
    <script>
      document.querySelector('#button5').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [5], {});
      };
    </script><br>
    <button id='button6'>6</button>
    <script>
      document.querySelector('#button6').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [6], {});
      };
    </script>
    <button id='button7'>7</button>
    <script>
      document.querySelector('#button7').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [7], {});
      };
    </script>
    <button id='button8'>8</button>
    <script>
      document.querySelector('#button8').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [8], {});
      };
    </script><br>
    <button id='buttonempezar'>Yo empiezo</button>
    <script>
      document.querySelector('#buttonempezar').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.empezar', [], {});
      };
    </script>
    '''))
#Creamos una nueva instancia de juego
mijuego = juego_interactivo(jugador_dinamico)
 
output.register_callback('notebook.updatelist',mijuego.turno_jugador1)
output.register_callback('notebook.empezar',mijuego.comenzar_juego)

  | | 
  | | 
  |X| 
-+-+-+-+-+
  | | 
  | | 
 O|X| 
-+-+-+-+-+
  | | 
  |X| 
 O|X| 
-+-+-+-+-+
  |O| 
  |X| 
 O|X| 
-+-+-+-+-+
  |O| 
 X|X| 
 O|X| 
-+-+-+-+-+
  |O| 
 X|X|O
 O|X| 
-+-+-+-+-+
  |O| 
 X|X|O
 O|X|X
-+-+-+-+-+
 O|O| 
 X|X|O
 O|X|X
-+-+-+-+-+
 O|O|X
 X|X|O
 O|X|X
-+-+-+-+-+
Empate: No hay ganador


# 6. Mejorar Optimizacion

In [0]:
def reflejo_x_conf(configuracion):
  matriz_conf = np.reshape(np.array(configuracion),(3,3))
  nueva_conf = np.zeros((3,3),dtype=int)
  nueva_conf[0,:] = matriz_conf[2,:]
  nueva_conf[1,:] = matriz_conf[1,:]
  nueva_conf[2,:] = matriz_conf[0,:]
  return (list(np.reshape(nueva_conf,9)))

def reflejo_y_conf(configuracion):
  matriz_conf = np.reshape(np.array(configuracion),(3,3))
  nueva_conf = np.zeros((3,3),dtype=int)
  nueva_conf[:,0] = matriz_conf[:,2]
  nueva_conf[:,1] = matriz_conf[:,1]
  nueva_conf[:,2] = matriz_conf[:,0]
  return (list(np.reshape(nueva_conf,9)))

def traspuesta_conf(configuracion):
  matriz_conf = np.reshape(np.array(configuracion),(3,3))
  nueva_conf = np.transpose(matriz_conf)
  return (list(np.reshape(nueva_conf,9)))

def reflejo_x_accion(accion):
  if accion < 3:
    return accion + 6
  if accion > 5:
    return accion - 6
  return accion
  
def reflejo_y_accion(accion):
  if (accion % 3 == 0):
    return accion + 2
  if (accion % 3 == 2):
    return accion - 2
  return accion

def traspuesta_accion(accion):
  i = accion % 3
  j = accion / 3
  return i*3+j

In [0]:
class MonteCarlo_mejorado:
  def __init__(self, jugador_estatico, jugador_dinamico, gamma = 0.8, alpha = 100):
    self.jugador_estatico = jugador_estatico
    self.jugador_dinamico = jugador_dinamico
    self. gamma = gamma
    self.alpha = alpha
    
  def modificar_valor(self,estado,accion,exponente,precio):
    indice = tuple(estado + [accion,])
    error = precio * (self.gamma ** exponente) - self.jugador_dinamico.valor_estado_accion[indice] 
    self.jugador_dinamico.valor_estado_accion[indice] += error/self.alpha

    
  def optimizar_jugador_dinamico(self,iteraciones = 1000, epsilon = 100):
    #Iterar sobre el total de iteraciones
    for i in xrange(iteraciones):
      
      #1. Decidir quien inicia
      quien_inicia = random.choice([1,2])
      
      #2.Modificar azar jugador dinamico
      self.jugador_dinamico.epsilon = epsilon / (1. + i)
      
      #3. Crear un juego nuevo
      nuevo_juego = juego(self.jugador_estatico, self.jugador_dinamico, quien_inicia)
    
      #4. Ejercer un juego
      [ganador,estados,acciones] = nuevo_juego.jugar()
      
      precio = 0
    
      if ganador == 2:
        precio = 1
      elif ganador == 1:
        precio = -1
             
      for paso,(estado,accion) in enumerate(zip(estados,acciones)):
        exponente = len(acciones) - paso
        
        self.modificar_valor(estado,accion,exponente,precio)
        self.modificar_valor(traspuesta_conf(estado),traspuesta_accion(accion),exponente,precio)
        
        #Apply symmetries
        estado2 = reflejo_x_conf(estado)
        accion2 = reflejo_x_accion(accion)
        self.modificar_valor(estado2,accion2,exponente,precio)
        self.modificar_valor(traspuesta_conf(estado2),traspuesta_accion(accion2),exponente,precio)
        
        estado3 = reflejo_y_conf(estado)
        accion3 = reflejo_y_accion(accion)
        self.modificar_valor(estado3,accion3,exponente,precio)
        self.modificar_valor(traspuesta_conf(estado3),traspuesta_accion(accion3),exponente,precio)
        
        estado4 = reflejo_x_conf(reflejo_y_conf(estado))
        accion4 = reflejo_x_accion(reflejo_y_accion(accion))
        self.modificar_valor(estado4,accion4,exponente,precio)
        self.modificar_valor(traspuesta_conf(estado4),traspuesta_accion(accion4),exponente,precio)
        

## Segunda Optimización

In [0]:
#Crear un valor estado_accion vacio
valor_estado_accion = np.zeros((3,)*9+(9,))

#Crear nuevo jugador dinamico
jugador_dinamico_2 = jugador(valor_estado_accion,epsilon = 1)

#Definir factor descuento
gamma = 0.8
alpha = 100

#Crear Monte Carlo
MC = MonteCarlo_mejorado(jugador_estatico, jugador_dinamico_2, gamma,alpha)

#Definir Iteraciones
iteraciones = 100000

#Aplicar Monte Carlo
MC.optimizar_jugador_dinamico(iteraciones, epsilon = 100)

## Juego Interactivo

In [49]:
display.display(display.HTML('''
    Posiciones del juego, tu eres X, yo soy O: <br>
    <button id='button0'>0</button>
    <script>
      document.querySelector('#button0').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [0], {});
      };
    </script>
    <button id='button1'>1</button>
    <script>
      document.querySelector('#button1').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [1], {});
      };
    </script>
    <button id='button2'>2</button>
    <script>
      document.querySelector('#button2').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [2], {});
      };
    </script><br>
    <button id='button3'>3</button>
    <script>
      document.querySelector('#button3').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [3], {});
      };
    </script>
    <button id='button4'>4</button>
    <script>
      document.querySelector('#button4').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [4], {});
      };
    </script>
    <button id='button5'>5</button>
    <script>
      document.querySelector('#button5').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [5], {});
      };
    </script><br>
    <button id='button6'>6</button>
    <script>
      document.querySelector('#button6').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [6], {});
      };
    </script>
    <button id='button7'>7</button>
    <script>
      document.querySelector('#button7').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [7], {});
      };
    </script>
    <button id='button8'>8</button>
    <script>
      document.querySelector('#button8').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [8], {});
      };
    </script><br>
    <button id='buttonempezar'>Yo empiezo</button>
    <script>
      document.querySelector('#buttonempezar').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.empezar', [], {});
      };
    </script>
    '''))
#Creamos una nueva instancia de juego
mijuego = juego_interactivo(jugador_dinamico_2)
 
output.register_callback('notebook.updatelist',mijuego.turno_jugador1)
output.register_callback('notebook.empezar',mijuego.comenzar_juego)

# Mejorar Jugador 

In [0]:
class jugador_mejorado: 
  def __init__(self, valor_estado_accion,epsilon = 1):
    """
    Representa a un jugador que decide tomar una accion de acuerdo al estado 
    para maximizar su función de valor
    Mejora el azar. Si reconoce una accion ganadora, la toma
    
    @param[in] valor_estado_accion: Arreglo de tamaño (3,3,3,3,3,3,3,3,3,9), 
               donde las primeras casillas especifican el estado y la ultima la 
               acción a tomar. Se asume que 1 corresponde a una ficha del otro
               jugador y 2 corresponde a una ficha de este jugador y 0 a una 
               ficha vacia
               
    @param[in] epsilon: es el componente aleatorio en la seleccion de la mejor
               accion. Debe ser positivo. Por defecto se asume totalmente
               aleatorio (epsilon = 1)
               
    @member    valor_estado_accion
    
    @member    epsilon
    """
    self.valor_estado_accion = copy.deepcopy(valor_estado_accion)
    self.epsilon = epsilon
  
  def accion_ganadora(self,estado,posibles_acciones):
    acciones_ganadoras = []
    mis_jugadas = 1*(np.array(estado)>1)
    for accion in posibles_acciones:
      futura_jugada = copy.deepcopy(mis_jugadas)
      futura_jugada[accion] = 1
      futura_jugada = np.reshape(futura_jugada,(3,3))
      #Sumar vertical
      sumas = np.sum(futura_jugada,0)
      #Sumar horizontal
      sumas = np.append(sumas, np.sum(futura_jugada,1))
      #Sumar diagonal principal
      sumas = np.append(sumas,futura_jugada[0,0]+futura_jugada[1,1]+futura_jugada[2,2])
      #Sumar diagonal secundaria
      sumas = np.append(sumas,futura_jugada[2,0]+futura_jugada[1,1]+futura_jugada[0,2])
      ganancias = np.sum(sumas > 2)
      if ganancias > 0:
        acciones_ganadoras.append(accion)
    return acciones_ganadoras
  
  def accion_optima(self, estado, posibles_acciones):
    """
    Esta función recibe un estado y una lista de posibles acciones y retorna la 
    accion optima para el jugador, usando el valor_estado_accion y el azar.
    
    @param[in] estado: Lista de 9 numeros con valores en las casillas en [0,1,2]
               donde 1 corresponde a una ficha del otro
               jugador y 2 corresponde a una ficha de este jugador y 0 a una 
               ficha vacia
               
    @param[in] posibles_acciones: Lista de numeros del 0 al 8 de casillas que 
               están vacias. Se asume que no está vacia
                              
    @return    mejor_accion: Numero del 0 al 8 que identifica la accion optima 
               de este jugador, que pertenece al vecotr de posibles acciones
    """
    # Primero revisa si hay acciones ganadoras
    acciones_ganadoras = self.accion_ganadora(estado,posibles_acciones)
    
    if len(acciones_ganadoras)>0:
      mejor_accion = random.choice(acciones_ganadoras)
      return mejor_accion
      
      
    #1. Decidir si se toma la accion al azar o se optimiza
    # Para esto generamos un número aleatorio. Si menor a epsilon * aleatoriedad
    # la decision es al azar, si no encontramos la mejor en valor_acciones
      
    u = random.random()
    
    #2. Si es al azar,  
    if u < self.epsilon:
      #3.1 Tome una muestra de posibles_acciones
      mejor_accion = random.choice(posibles_acciones)
    
    #3. Si no, 
    else:
      #4.1 Extraer la fila de valor_estado_accion que corresponde a estado 
      # y que contiene solo las acciones posibles
      valor_acciones = self.valor_estado_accion[tuple(estado)] [posibles_acciones]

      #4.2 Encuentre el indice con mayor valor en la fila que extraimos en 1.
      mejor_valor = np.amax(valor_acciones)
      mejores_indices = np.where(valor_acciones == mejor_valor )[0]
      
      #4.3 Traducir este indice en el vector de acciones posibles 
      mejor_accion = posibles_acciones[random.choice(mejor_indice)] 
      
    #4. Retornar la mejor accion
    return mejor_accion


# Tercera Optimizacion

In [0]:
#Crear jugador estatico como aleatorio
jugador_estatico_2 = jugador_mejorado([],epsilon = 1)

#Crear un valor estado_accion vacio
valor_estado_accion = np.zeros((3,)*9+(9,))

#Crear jugador dinamico
jugador_dinamico_3 = jugador(valor_estado_accion,epsilon = 1)

#Definir factor descuento
gamma = 1
alpha = 100

#Crear Monte Carlo
MC = MonteCarlo_mejorado(jugador_estatico_2, jugador_dinamico_3, gamma,alpha)

#Definir Iteraciones
iteraciones = 100000

#Aplicar Monte Carlo
MC.optimizar_jugador_dinamico(iteraciones, epsilon = 100)

## Juego Interacttivo

In [84]:
display.display(display.HTML('''
    Posiciones del juego, tu eres X, yo soy O: <br>
    <button id='button0'>0</button>
    <script>
      document.querySelector('#button0').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [0], {});
      };
    </script>
    <button id='button1'>1</button>
    <script>
      document.querySelector('#button1').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [1], {});
      };
    </script>
    <button id='button2'>2</button>
    <script>
      document.querySelector('#button2').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [2], {});
      };
    </script><br>
    <button id='button3'>3</button>
    <script>
      document.querySelector('#button3').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [3], {});
      };
    </script>
    <button id='button4'>4</button>
    <script>
      document.querySelector('#button4').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [4], {});
      };
    </script>
    <button id='button5'>5</button>
    <script>
      document.querySelector('#button5').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [5], {});
      };
    </script><br>
    <button id='button6'>6</button>
    <script>
      document.querySelector('#button6').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [6], {});
      };
    </script>
    <button id='button7'>7</button>
    <script>
      document.querySelector('#button7').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [7], {});
      };
    </script>
    <button id='button8'>8</button>
    <script>
      document.querySelector('#button8').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [8], {});
      };
    </script><br>
    <button id='buttonempezar'>Yo empiezo</button>
    <script>
      document.querySelector('#buttonempezar').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.empezar', [], {});
      };
    </script>
    '''))
#Creamos una nueva instancia de juego
jugador_dinamico_3.epsilon = 0
mijuego = juego_interactivo(jugador_dinamico_3)
 
output.register_callback('notebook.updatelist',mijuego.turno_jugador1)
output.register_callback('notebook.empezar',mijuego.comenzar_juego)